### Emprical Asset Pricing - Problem Set 1

In [2]:
# Packages
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wrds
from datetime import datetime, timedelta
from pandas.tseries.offsets import MonthEnd

# Setups
pd.set_option("display.max_rows", 100)

In [11]:
# Set Up WRDS connection
db = wrds.Connection(wrds_username='vince_solis') # make sure to change the username. 

Loading library list...


ObjectNotExecutableError: Not an executable object: '\nWITH pgobjs AS (\n    -- objects we care about - tables, views, foreign tables, partitioned tables\n    SELECT oid, relnamespace, relkind\n    FROM pg_class\n    WHERE relkind = ANY (ARRAY[\'r\'::"char", \'v\'::"char", \'f\'::"char", \'p\'::"char"])\n),\nschemas AS (\n    -- schemas we have usage on that represent products\n    SELECT nspname AS schemaname,\n        pg_namespace.oid,\n        array_agg(DISTINCT relkind) AS relkind_a\n    FROM pg_namespace\n    JOIN pgobjs ON pg_namespace.oid = relnamespace\n    WHERE nspname !~ \'(^pg_)|(_old$)|(_new$)|(information_schema)\'\n        AND has_schema_privilege(nspname, \'USAGE\') = TRUE\n    GROUP BY nspname, pg_namespace.oid\n)\nSELECT schemaname\nFROM schemas\nWHERE relkind_a != ARRAY[\'v\'::"char"] -- any schema except only views\nUNION\n-- schemas w/ views (aka "friendly names") that reference accessable product tables\nSELECT nv.schemaname\nFROM schemas nv\nJOIN pgobjs v ON nv.o

### 0. Data Extraction and Cleaning

In [4]:
data = pd.read_csv('./data/crsp_1945-2023.csv', index_col=0)
# data['caldt'] = pd.to_datetime(data['caldt'])
# data.set_index(data['caldt'])
data.head()

,vwretd,vwretx
caldt,,
1945-01-31,0.020218,0.018951
1945-02-28,0.064477,0.059894
1945-03-31,-0.039177,-0.043164
1945-04-30,0.078232,0.076981
1945-05-31,0.018185,0.012439


In [10]:
ff = pd.read_csv('./data/F-F_Research_Data_Factors.csv', skiprows=3, infer_datetime_format=True, index_col=0)
ff.index = pd.to_datetime(ff.index)

C:\Users\ConquerV\AppData\Local\Temp\ipykernel_15236\4036697621.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  ff = pd.read_csv('./data/F-F_Research_Data_Factors.csv', skiprows=3, infer_datetime_format=True, index_col=0)
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_15236\4036697621.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ff.index = pd.to_datetime(ff.index)


DateParseError: month must be in 1..12: 192607, at position 0

### 1. Time-Series Predictability of returns and dividend growth

In [10]:
data['dividend'] = data['vwretd'] - data['vwretx']

Aggregating dividends